In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('instance').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/23 22:58:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Partition Spark Data Frame by single column
* `partitionBy` kwarg of `DataFrameWriter.write.parquet`
* `partitionBy` DataFrame function 

In [9]:
json_path = '/Users/adhoc/git/retail_db/json/orders'

orders = spark.read.json(json_path)
orders.show()
orders.dtypes

+-----------------+--------------------+--------+---------------+
|order_customer_id|          order_date|order_id|   order_status|
+-----------------+--------------------+--------+---------------+
|            11599|2013-07-25T00:00:...|       1|         CLOSED|
|              256|2013-07-25T00:00:...|       2|PENDING_PAYMENT|
|            12111|2013-07-25T00:00:...|       3|       COMPLETE|
|             8827|2013-07-25T00:00:...|       4|         CLOSED|
|            11318|2013-07-25T00:00:...|       5|       COMPLETE|
|             7130|2013-07-25T00:00:...|       6|       COMPLETE|
|             4530|2013-07-25T00:00:...|       7|       COMPLETE|
|             2911|2013-07-25T00:00:...|       8|     PROCESSING|
|             5657|2013-07-25T00:00:...|       9|PENDING_PAYMENT|
|             5648|2013-07-25T00:00:...|      10|PENDING_PAYMENT|
|              918|2013-07-25T00:00:...|      11| PAYMENT_REVIEW|
|             1837|2013-07-25T00:00:...|      12|         CLOSED|
|         

[('order_customer_id', 'bigint'),
 ('order_date', 'string'),
 ('order_id', 'bigint'),
 ('order_status', 'string')]

In [11]:
from pyspark.sql.functions import date_format

orders. \
    withColumn('order_date', date_format('order_date', 'yyyyMMdd')). \
    show()

+-----------------+----------+--------+---------------+
|order_customer_id|order_date|order_id|   order_status|
+-----------------+----------+--------+---------------+
|            11599|  20130725|       1|         CLOSED|
|              256|  20130725|       2|PENDING_PAYMENT|
|            12111|  20130725|       3|       COMPLETE|
|             8827|  20130725|       4|         CLOSED|
|            11318|  20130725|       5|       COMPLETE|
|             7130|  20130725|       6|       COMPLETE|
|             4530|  20130725|       7|       COMPLETE|
|             2911|  20130725|       8|     PROCESSING|
|             5657|  20130725|       9|PENDING_PAYMENT|
|             5648|  20130725|      10|PENDING_PAYMENT|
|              918|  20130725|      11| PAYMENT_REVIEW|
|             1837|  20130725|      12|         CLOSED|
|             9149|  20130725|      13|PENDING_PAYMENT|
|             9842|  20130725|      14|     PROCESSING|
|             2568|  20130725|      15|       CO

In [41]:
parquet_path = '/Users/adhoc/git/retail_db/parquet/orders'

orders. \
    withColumn('order_date', date_format('order_date', 'yyyyMMdd')). \
    coalesce(1). \
    write. \
    mode('overwrite'). \
    partitionBy('order_date'). \
    parquet(parquet_path)

In [42]:
spark.read.parquet(parquet_path).show()
len(spark.read.parquet(parquet_path).inputFiles()), \
spark.read.parquet(parquet_path).inputFiles()[:10], \
spark.read.parquet(parquet_path).dtypes

+-----------------+--------+---------------+----------+
|order_customer_id|order_id|   order_status|order_date|
+-----------------+--------+---------------+----------+
|             6471|   15793|       COMPLETE|  20131103|
|             5323|   15794|     PROCESSING|  20131103|
|            10096|   15795|         CLOSED|  20131103|
|            11665|   15796|       COMPLETE|  20131103|
|             6249|   15797|PENDING_PAYMENT|  20131103|
|            10736|   15798|       COMPLETE|  20131103|
|             5475|   15799|       COMPLETE|  20131103|
|             7417|   15800|     PROCESSING|  20131103|
|             4021|   15801|       COMPLETE|  20131103|
|             2284|   15802|         CLOSED|  20131103|
|             1015|   15803|     PROCESSING|  20131103|
|              208|   15804|       COMPLETE|  20131103|
|             3708|   15805|        PENDING|  20131103|
|            10239|   15806|PENDING_PAYMENT|  20131103|
|             4709|   15807|PENDING_PAYMENT|  20

(364,
 ['file:///Users/adhoc/git/retail_db/parquet/orders/order_date=20140204/part-00000-dce80bef-67ab-4b1d-adcc-ac05932d939a.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_date=20130809/part-00000-dce80bef-67ab-4b1d-adcc-ac05932d939a.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_date=20140502/part-00000-dce80bef-67ab-4b1d-adcc-ac05932d939a.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_date=20140614/part-00000-dce80bef-67ab-4b1d-adcc-ac05932d939a.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_date=20140412/part-00000-dce80bef-67ab-4b1d-adcc-ac05932d939a.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_date=20140604/part-00000-dce80bef-67ab-4b1d-adcc-ac05932d939a.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_date=20140402/part-00000-dce80bef-67ab-4b1d-adcc-ac05932d939a.c000.snappy.parquet',


In [43]:
orders. \
    withColumn('order_month', date_format('order_date', 'yyyyMM')). \
    coalesce(1). \
    write. \
    mode('overwrite'). \
    parquet(parquet_path, partitionBy='order_month')

In [44]:
spark.read.parquet(parquet_path).show()
len(spark.read.parquet(parquet_path).inputFiles()), \
spark.read.parquet(parquet_path).inputFiles()[:10], \
spark.read.parquet(parquet_path).dtypes

+-----------------+--------------------+--------+---------------+-----------+
|order_customer_id|          order_date|order_id|   order_status|order_month|
+-----------------+--------------------+--------+---------------+-----------+
|             8987|2013-11-01T00:00:...|   15488|PENDING_PAYMENT|     201311|
|             5359|2013-11-01T00:00:...|   15489|PENDING_PAYMENT|     201311|
|            10149|2013-11-01T00:00:...|   15490|       COMPLETE|     201311|
|            10635|2013-11-01T00:00:...|   15491|        ON_HOLD|     201311|
|             7784|2013-11-01T00:00:...|   15492|PENDING_PAYMENT|     201311|
|             1104|2013-11-01T00:00:...|   15493|        ON_HOLD|     201311|
|             7313|2013-11-01T00:00:...|   15494|     PROCESSING|     201311|
|             7067|2013-11-01T00:00:...|   15495|         CLOSED|     201311|
|            12153|2013-11-01T00:00:...|   15496|PENDING_PAYMENT|     201311|
|            11115|2013-11-01T00:00:...|   15497|PENDING_PAYMENT

(13,
 ['file:///Users/adhoc/git/retail_db/parquet/orders/order_month=201403/part-00000-72badc54-5e17-47aa-8e30-638d5b5f867c.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_month=201310/part-00000-72badc54-5e17-47aa-8e30-638d5b5f867c.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_month=201406/part-00000-72badc54-5e17-47aa-8e30-638d5b5f867c.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_month=201311/part-00000-72badc54-5e17-47aa-8e30-638d5b5f867c.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_month=201308/part-00000-72badc54-5e17-47aa-8e30-638d5b5f867c.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_month=201407/part-00000-72badc54-5e17-47aa-8e30-638d5b5f867c.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/order_month=201309/part-00000-72badc54-5e17-47aa-8e30-638d5b5f867c.c000.snappy.parquet',
  'file:

## Partition Spark Data Frame by multipe columns (composite partitioning)

In [46]:
from pyspark.sql.functions import date_format

parquet_path = '/Users/adhoc/git/retail_db/parquet/orders'

orders. \
    withColumn('year', date_format('order_date', 'yyyy')). \
    withColumn('month', date_format('order_date', 'MM')). \
    withColumn('day', date_format('order_date', 'dd')). \
    coalesce(1). \
    write. \
    mode('overwrite'). \
    parquet(parquet_path, partitionBy=['year', 'month', 'day'])

In [47]:
spark.read.parquet(parquet_path).show()
len(spark.read.parquet(parquet_path).inputFiles()), \
spark.read.parquet(parquet_path).inputFiles()[:10], \
spark.read.parquet(parquet_path).dtypes

+-----------------+--------------------+--------+---------------+----+-----+---+
|order_customer_id|          order_date|order_id|   order_status|year|month|day|
+-----------------+--------------------+--------+---------------+----+-----+---+
|             6471|2013-11-03T00:00:...|   15793|       COMPLETE|2013|   11|  3|
|             5323|2013-11-03T00:00:...|   15794|     PROCESSING|2013|   11|  3|
|            10096|2013-11-03T00:00:...|   15795|         CLOSED|2013|   11|  3|
|            11665|2013-11-03T00:00:...|   15796|       COMPLETE|2013|   11|  3|
|             6249|2013-11-03T00:00:...|   15797|PENDING_PAYMENT|2013|   11|  3|
|            10736|2013-11-03T00:00:...|   15798|       COMPLETE|2013|   11|  3|
|             5475|2013-11-03T00:00:...|   15799|       COMPLETE|2013|   11|  3|
|             7417|2013-11-03T00:00:...|   15800|     PROCESSING|2013|   11|  3|
|             4021|2013-11-03T00:00:...|   15801|       COMPLETE|2013|   11|  3|
|             2284|2013-11-0

(364,
 ['file:///Users/adhoc/git/retail_db/parquet/orders/year=2014/month=03/day=30/part-00000-438cc701-3042-4014-89a6-73b52ecf12af.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2014/month=04/day=07/part-00000-438cc701-3042-4014-89a6-73b52ecf12af.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2013/month=11/day=10/part-00000-438cc701-3042-4014-89a6-73b52ecf12af.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2013/month=11/day=23/part-00000-438cc701-3042-4014-89a6-73b52ecf12af.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2014/month=04/day=24/part-00000-438cc701-3042-4014-89a6-73b52ecf12af.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2013/month=12/day=21/part-00000-438cc701-3042-4014-89a6-73b52ecf12af.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2013/month=08/day=06/part-00000-438cc701-3042-401

In [56]:
orders. \
    withColumn('year', date_format('order_date', 'yyyy')). \
    withColumn('month', date_format('order_date', 'MM')). \
    withColumn('day', date_format('order_date', 'dd')). \
    coalesce(1). \
    write. \
    partitionBy('year', 'month', 'day'). \
    mode('overwrite'). \
    parquet(parquet_path, mode='overwrite')

In [57]:
spark.read.parquet(parquet_path).show()
len(spark.read.parquet(parquet_path).inputFiles()), \
spark.read.parquet(parquet_path).inputFiles()[:10], \
spark.read.parquet(parquet_path).dtypes

+-----------------+--------------------+--------+---------------+----+-----+---+
|order_customer_id|          order_date|order_id|   order_status|year|month|day|
+-----------------+--------------------+--------+---------------+----+-----+---+
|             6471|2013-11-03T00:00:...|   15793|       COMPLETE|2013|   11|  3|
|             5323|2013-11-03T00:00:...|   15794|     PROCESSING|2013|   11|  3|
|            10096|2013-11-03T00:00:...|   15795|         CLOSED|2013|   11|  3|
|            11665|2013-11-03T00:00:...|   15796|       COMPLETE|2013|   11|  3|
|             6249|2013-11-03T00:00:...|   15797|PENDING_PAYMENT|2013|   11|  3|
|            10736|2013-11-03T00:00:...|   15798|       COMPLETE|2013|   11|  3|
|             5475|2013-11-03T00:00:...|   15799|       COMPLETE|2013|   11|  3|
|             7417|2013-11-03T00:00:...|   15800|     PROCESSING|2013|   11|  3|
|             4021|2013-11-03T00:00:...|   15801|       COMPLETE|2013|   11|  3|
|             2284|2013-11-0

(364,
 ['file:///Users/adhoc/git/retail_db/parquet/orders/year=2014/month=06/day=18/part-00000-711cc3a5-fcdb-4f77-91bf-0674067d3552.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2013/month=08/day=01/part-00000-711cc3a5-fcdb-4f77-91bf-0674067d3552.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2014/month=03/day=16/part-00000-711cc3a5-fcdb-4f77-91bf-0674067d3552.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2014/month=02/day=08/part-00000-711cc3a5-fcdb-4f77-91bf-0674067d3552.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2013/month=09/day=02/part-00000-711cc3a5-fcdb-4f77-91bf-0674067d3552.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2013/month=08/day=18/part-00000-711cc3a5-fcdb-4f77-91bf-0674067d3552.c000.snappy.parquet',
  'file:///Users/adhoc/git/retail_db/parquet/orders/year=2013/month=12/day=15/part-00000-711cc3a5-fcdb-4f7

## Partition pruning

In [64]:
spark.read.parquet(parquet_path).count()


68883

In [63]:
#partition pruning
spark.read.parquet(parquet_path).filter('year=2013').count()

30662

In [67]:
spark.read.parquet(parquet_path).createOrReplaceTempView('orders')
spark.sql('SHOW tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |   orders|       true|
+---------+---------+-----------+



In [77]:
spark.sql("""
    select count(1)
    from orders
    where year=2013
""").show()

# partition pruning in explain plan -> PartitionFilters: [isnotnull(year#1853), (year#1853 = 2013)]
spark.sql("""
    select count(1)
    from orders
    where year=2013
""").explain()

+--------+
|count(1)|
+--------+
|   30662|
+--------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(1)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=1278]
      +- HashAggregate(keys=[], functions=[partial_count(1)])
         +- Project
            +- FileScan parquet [year#1853,month#1854,day#1855] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/Users/adhoc/git/retail_db/parquet/orders], PartitionFilters: [isnotnull(year#1853), (year#1853 = 2013)], PushedFilters: [], ReadSchema: struct<>




In [76]:
#getting spark URL -> localhost:4040
spark.sparkContext.uiWebUrl

'http://192.168.200.166:4040'